# Semantische Kernel met OpenBnB MCP Server-integratie

Dit notebook laat zien hoe je de Semantische Kernel kunt gebruiken met de daadwerkelijke OpenBnB MCP-server om te zoeken naar echte Airbnb-accommodaties met behulp van MCPStdioPlugin. Voor toegang tot LLM wordt gebruikgemaakt van Azure AI Foundry. Om je omgevingsvariabelen in te stellen, kun je de [Setup Les](/00-course-setup/README.md) volgen.


## Importeer de benodigde pakketten


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## De MCP Plugin-verbinding maken

We gaan verbinding maken met de [OpenBnB MCP-server](https://github.com/openbnb-org/mcp-server-airbnb) met behulp van MCPStdioPlugin. Deze server biedt zoekfunctionaliteit voor Airbnb via het @openbnb/mcp-server-airbnb-pakket.


## De client maken

In dit voorbeeld gebruiken we Azure AI Foundry voor onze toegang tot LLM. Zorg ervoor dat je omgevingsvariabelen correct zijn ingesteld.


## Omgevingsconfiguratie

Configureer Azure OpenAI-instellingen. Zorg ervoor dat je de volgende omgevingsvariabelen hebt ingesteld:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Begrijpen van de OpenBnB MCP-integratie

Dit notebook maakt verbinding met de **echte OpenBnB MCP-server** die daadwerkelijke Airbnb-zoekfunctionaliteit biedt.

### Hoe het werkt:

1. **MCPStdioPlugin**: Maakt gebruik van standaard invoer/uitvoer communicatie met de MCP-server
2. **Echte NPM-pakket**: Downloadt en voert `@openbnb/mcp-server-airbnb` uit via npx
3. **Live Data**: Geeft echte Airbnb-eigendomsgegevens terug via hun API's
4. **Functieontdekking**: De agent ontdekt automatisch beschikbare functies van de MCP-server

### Beschikbare functies:

De OpenBnB MCP-server biedt doorgaans de volgende functies:
- **search_listings** - Zoek naar Airbnb-eigendommen op locatie en criteria
- **get_listing_details** - Verkrijg gedetailleerde informatie over specifieke eigendommen
- **check_availability** - Controleer beschikbaarheid voor specifieke datums
- **get_reviews** - Haal beoordelingen op voor eigendommen
- **get_host_info** - Verkrijg informatie over de hosts van eigendommen

### Vereisten:

- **Node.js** geïnstalleerd op je systeem
- **Internetverbinding** om het MCP-serverpakket te downloaden
- **NPX** beschikbaar (wordt meegeleverd met Node.js)

### Verbinding testen:

Je kunt de MCP-server handmatig testen door het volgende uit te voeren:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Hiermee wordt de OpenBnB MCP-server gedownload en gestart, waarmee Semantic Kernel verbinding maakt om echte Airbnb-gegevens te verkrijgen.


## Het uitvoeren van de Agent met de OpenBnB MCP-server

Nu gaan we de AI Agent uitvoeren die verbinding maakt met de OpenBnB MCP-server om echte Airbnb-accommodaties in Stockholm te zoeken voor 2 volwassenen en 1 kind. Pas gerust de `user_inputs` lijst aan om de zoekcriteria te wijzigen.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Samenvatting  
Gefeliciteerd! Je hebt met succes een AI-agent gebouwd die integreert met het zoeken naar accommodaties in de echte wereld via het Model Context Protocol (MCP):

Gebruikte technologieën:  
- Semantic Kernel - Voor het bouwen van intelligente agents met Azure OpenAI  
- Azure AI Foundry - Voor LLM-mogelijkheden en chatafhandeling  
- MCP (Model Context Protocol) - Voor gestandaardiseerde toolintegratie  
- OpenBnB MCP Server - Voor echte zoekfunctionaliteit van Airbnb  
- Node.js/NPX - Voor het draaien van de externe MCP-server  

Wat je hebt geleerd:  
- MCP-integratie: Het verbinden van Semantic Kernel-agents met externe MCP-servers  
- Toegang tot realtime gegevens: Het zoeken naar echte Airbnb-accommodaties via live API's  
- Protocolcommunicatie: Het gebruik van stdio-communicatie tussen de agent en de MCP-server  
- Functieontdekking: Automatisch ontdekken van beschikbare functies van MCP-servers  
- Streamingreacties: Het vastleggen en loggen van functieaanroepen in realtime  
- HTML-weergave: Het formatteren van agentreacties met gestileerde tabellen en interactieve displays  

Volgende stappen:  
- Integreer extra MCP-servers (weer, vluchten, restaurants)  
- Bouw een multi-agent systeem dat MCP- en A2A-protocollen combineert  
- Maak aangepaste MCP-servers voor je eigen gegevensbronnen  
- Implementeer een blijvend gespreksgeheugen over sessies heen  
- Zet de agent in op Azure Functions met MCP-serverorkestratie  
- Voeg gebruikersauthenticatie en boekingsmogelijkheden toe  

Belangrijke voordelen van MCP-architectuur:  
- Standaardisatie: Universeel protocol voor het verbinden van AI-agents met externe tools  
- Realtime gegevens: Toegang tot live, actuele informatie van verschillende diensten  
- Uitbreidbaarheid: Gemakkelijke integratie van nieuwe gegevensbronnen en tools  
- Interoperabiliteit: Werkt met verschillende AI-frameworks en agentplatforms  
- Scheiding van verantwoordelijkheden: Duidelijke scheiding tussen AI-logica en externe gegevensbronnen  



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we ons best doen om nauwkeurigheid te garanderen, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
